<a href="https://colab.research.google.com/github/imusicmash/stanford_llm_python/blob/main/StanfordLLMClassWeek5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install sentence-transformers
!pip install langchain pypdf langchain-openai #tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

# RAG

In [ ]:
!pip install llama-index --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 113.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pypdf

In [ ]:
!wget https://www.goldmansachs.com/intelligence/pages/gs-research/2024-us-equity-outlook-all-you-had-to-do-was-stay/report.pdf

--2024-02-29 05:16:00--  https://www.goldmansachs.com/intelligence/pages/gs-research/2024-us-equity-outlook-all-you-had-to-do-was-stay/report.pdf
Resolving www.goldmansachs.com (www.goldmansachs.com)... 104.98.131.73
Connecting to www.goldmansachs.com (www.goldmansachs.com)|104.98.131.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491250 (480K) [application/pdf]
Saving to: ‘report.pdf’

report.pdf          100%[===================>] 479.74K  --.-KB/s    in 0.05s   

2024-02-29 05:16:00 (8.91 MB/s) - ‘report.pdf’ saved [491250/491250]



In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')

# Routing

In [ ]:
# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, SummaryIndex
from llama_index.core import Settings

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = SimpleDirectoryReader("./").load_data("report.pdf")

# initialize settings (set chunk size)
Settings.chunk_size = 1024
nodes = Settings.node_parser.get_nodes_from_documents(documents)

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()


Loading files: 100%|██████████| 1/1 [00:00<00:00,  1.08file/s]


In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions related to the data source",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context related to the data source",
)

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
)


In [ ]:
response = query_engine.query("What is the 2024 outlook for US GDP?")
print(response)

The 2024 outlook for US GDP is forecasted to have above-consensus full-year GDP growth of 2.1%.


In [ ]:
response = query_engine.query("Summarize the document")
print(response)

The document is a comprehensive research report by Goldman Sachs providing insights and forecasts for the US equity market in 2024, particularly focusing on the S&P 500 index. It discusses the outlook for the market, emphasizing factors such as economic expansion, earnings growth, stable equity valuations, and the performance of mega-cap tech stocks. The report highlights potential investment opportunities, risks, and uncertainties in the market, including concerns about a potential recession, financial stability issues, and geopolitical tensions. It also delves into the impact of macroeconomic factors, interest rates, and investor behavior on equity markets. Additionally, the document analyzes the performance and outlook of the top seven mega-cap technology stocks compared to the rest of the index, addressing factors influencing their performance such as sales growth, margins, and hedge fund ownership levels. The report concludes with detailed disclosures and information related to va

# Sub Question Query Engine

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

documents = SimpleDirectoryReader("./").load_data("report.pdf")

# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    documents,
    use_async=True,
).as_query_engine()


# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="documents",
            description="Report",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

response = query_engine.query(
    "What is the outlook for the US economy?"
)

print(response)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  1.09file/s]


**********
Trace: index_construction
    |_embedding ->  1.050074 seconds
**********
Generated 2 sub questions.
[documents] Q: What are the current economic indicators for the US economy?
[documents] Q: What is the GDP growth forecast for the US economy?
[documents] A: The GDP growth forecast for the US economy is 2.1% in 2024.
[documents] A: The current economic indicators for the US economy include the fear of a recession among investors despite a 15% likelihood according to Goldman Sachs (compared to a 55% consensus from Bloomberg), concerns about commercial real estate impacting regional banks, upcoming antitrust rulings affecting Big Tech firms, the uncertainty surrounding the US presidential election, and geopolitical risks such as the Israel-Hamas War, the Russia-Ukraine War, and the US-China Trade War.
**********
Trace: query
    |_query ->  4.766648 seconds
      |_llm ->  0.98195 seconds
      |_sub_question ->  2.135969 seconds
        |_query ->  2.13563 seconds
          |

# Calling OpenAI AssistantAPI (Code interpreter)

In [ ]:
from llama_index.agent.openai import OpenAIAssistantAgent

agent = OpenAIAssistantAgent.from_new(
    name="Python agent",
    openai_tools=[{"type": "code_interpreter"}],
    instructions="You are an expert at writing python code to solve problems.",
    verbose=True
)

response = agent.chat(
    """Generate code to answer the following question:
    How much is the us population likely to grow to by 2030?
    Return and answer and the code used."""
)

In [ ]:
print(str(response))

Based on the simple linear growth model with an assumed annual growth rate of 0.5%, the estimated population of the United States by 2030 could grow to approximately 347.9 million people.

Please note that this calculation is based on an assumed growth rate and a simplistic model. For actual projections, one would need to use more complex demographic models and current data from authoritative sources.

Here is the Python code that was used to calculate this estimate:

```python
# Estimated population in 2020
population_2020 = 331000000  # 331 million

# Assuming an average annual growth rate of 0.5%
annual_growth_rate = 0.005  # 0.5%

# Number of years from 2020 to 2030
years = 10

# Calculate the estimated population for 2030 using the linear growth model
population_2030 = population_2020 * (1 + annual_growth_rate) ** years
population_2030
```

The result from the execution of this code is approximately 347,927,384 people.


# ReAct

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-02-29 05:17:06--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.04s   

2024-02-29 05:17:06 (45.5 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-02-29 05:17:06--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP 

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

**********
Trace: index_construction
    |_embedding ->  1.72652 seconds
    |_embedding ->  2.247811 seconds
    |_embedding ->  2.532917 seconds
    |_embedding ->  1.099217 seconds
**********
**********
Trace: index_construction
    |_embedding ->  1.333994 seconds
    |_embedding ->  1.637265 seconds
    |_embedding ->  1.27136 seconds
    |_embedding ->  1.583719 seconds
    |_embedding ->  0.497963 seconds
**********


In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    # context=context
)

In [ ]:
response = agent.chat(
    "Compare the risk of Uber and Lyft and return a table"
)
print(str(response))

Thought: I need to use a tool to help me compare the risks of Uber and Lyft in a structured format.
Action: uber_10k
Action Input: {'input': 'Please provide a comparison of the risks faced by Uber in 2021.'}
Observation: Uber faced various risks in 2021, including regulatory challenges related to acquisitions, the need to lower fares and offer incentives to remain competitive, potential driver supply constraints due to the COVID-19 pandemic, uncertainties surrounding the classification of drivers, and significant losses incurred since inception. Additionally, the impact of COVID-19 on market conditions, demand for mobility services, and the need to adapt to changing consumer behaviors posed risks to Uber's business operations and financial performance.
Thought: I need to gather information on the risks faced by Lyft in 2021 to compare with Uber's risks.
Action: lyft_10k
Action Input: {'input': 'Please provide a comparison of the risks faced by Lyft in 2021.'}
Observation: Lyft faced va

# MSFT AutoGen